In [2]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np

In [3]:
df_2000s = pd.read_csv("Spotify-2000.csv")

df_2000s.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59


In [4]:
df_2000s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Index                   1994 non-null   int64 
 1   Title                   1994 non-null   object
 2   Artist                  1994 non-null   object
 3   Top Genre               1994 non-null   object
 4   Year                    1994 non-null   int64 
 5   Beats Per Minute (BPM)  1994 non-null   int64 
 6   Energy                  1994 non-null   int64 
 7   Danceability            1994 non-null   int64 
 8   Loudness (dB)           1994 non-null   int64 
 9   Liveness                1994 non-null   int64 
 10  Valence                 1994 non-null   int64 
 11  Length (Duration)       1994 non-null   object
 12  Acousticness            1994 non-null   int64 
 13  Speechiness             1994 non-null   int64 
 14  Popularity              1994 non-null   int64 
dtypes: i

In [5]:
len(df_2000s["Top Genre"].unique())

149

In [6]:
df_2000s["Top Genre"].value_counts()

album rock           413
adult standards      123
dutch pop             88
alternative rock      86
dance pop             83
                    ... 
streektaal             1
indie anthem-folk      1
la pop                 1
ccm                    1
bebop                  1
Name: Top Genre, Length: 149, dtype: int64

In [7]:
df_2000s.drop(columns = ['Index', 'Title', 'Artist', 'Year'], inplace = True)

In [10]:
attributes = df_2000s.columns[1:]
for attribute in attributes:
    temp = df_2000s[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df_2000s[attribute][instance] = float(temp[instance].replace(',',''))
            

C:\Users\727mo\AppData\Local\Temp\ipykernel_6700\1556948572.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000s[attribute][instance] = float(temp[instance].replace(',',''))


In [12]:
genre = (df_2000s["Top Genre"].str.strip()).str.lower()

In [13]:
# multiclass classification
# function to split genre column

def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ", 1)
    for i in range(len(result)):
        if(len(result[i])>1):
            result[i] = [result[i][1]]
    return result.str.join('')

In [14]:
# loop until the genre cannot be split any further
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ", 1)).str.len())> 1):
    genre_m1 = genre_splitter(genre_m1)

In [15]:
len(genre_m1.unique())

68

In [16]:
genre_m1.value_counts()

rock           856
pop            333
standards      123
metal           93
indie           77
              ... 
laboratorio      1
holler           1
hardcore         1
trance           1
bebop            1
Name: Top Genre, Length: 68, dtype: int64

In [18]:
unique = genre_m1.unique()
to_remove = []

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 20:
        to_remove += [genre]
len(to_remove)

56

In [19]:
df_2000s['Top Genre'] = genre_m1
df_2000s

,Top Genre,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,standards,157,30,53,-14,11,68,201.0,94,3,71
1,rock,135,79,50,-11,17,81,207.0,17,7,39
2,hop,168,69,66,-9,7,52,341.0,2,17,69
3,metal,173,96,43,-4,3,37,269.0,0,4,76
4,rock,106,82,58,-5,10,87,256.0,1,3,59
...,...,...,...,...,...,...,...,...,...,...,...
1989,standards,94,21,70,-12,11,72,128.0,84,7,63
1990,standards,175,76,36,-8,76,95,136.0,73,6,69
1991,rock,168,80,53,-9,31,97,162.0,74,7,74
1992,bebop,174,26,45,-13,7,60,324.0,54,4,65
